In [31]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np 
import os
import pandas as pd

In [32]:
os.getcwd()
os.chdir('C://Users//Jaesung//Desktop//해오라기//합칠거')

In [33]:
data = pd.read_csv('전처리완료.csv',engine ='python')

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18398 entries, 0 to 18397
Data columns (total 11 columns):
안정성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_커피음료      18398 non-null int64
성장성_지표_구간_코드          18398 non-null int64
서비스_업종_코드_명_치킨집       18398 non-null int64
서비스_업종_코드_명_패스트푸드점    18398 non-null int64
서비스_업종_코드_명_분식집       18398 non-null int64
cluster               18398 non-null int64
점포수                   18398 non-null int64
중년층_직장인구_수            18398 non-null int64
총_직장인구_수              18398 non-null int64
과밀_지수_구간_코드           18398 non-null float64
dtypes: float64(1), int64(10)
memory usage: 1.5 MB


In [35]:
X = data[data.columns[data.columns!='과밀_지수_구간_코드']]
Y = data[data.columns[data.columns=='과밀_지수_구간_코드']]

In [36]:
data["과밀_지수_구간_코드"].value_counts()

0.0    6375
2.0    6101
1.0    5922
Name: 과밀_지수_구간_코드, dtype: int64

In [37]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

## modeling - SVM

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import StratifiedKFold

In [39]:
import warnings
warnings.filterwarnings(action='ignore') 

In [40]:
n_folds = 3

def cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    score = cross_validate(model, X_train, y_train, scoring=['accuracy'], cv=kf, return_train_score=True)
    return(score)


#    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))



In [41]:
model_svc = SVC(C=10, gamma=0.1, coef0=0, degree=3,  kernel='rbf')
#model_svc = SVC()


In [42]:
cv(model_svc)

{'fit_time': array([11.56607342, 12.0687654 , 11.34068012]),
 'score_time': array([3.06883812, 3.07478905, 2.97907233]),
 'test_accuracy': array([0.4077783 , 0.41309413, 0.41612302]),
 'train_accuracy': array([0.99603914, 0.9940601 , 0.99522478])}

In [43]:
model_svc.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
predictions  = model_svc.predict(X_test)

In [45]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_test, predictions, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, predictions))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

F-Score: 0.436
Accuracy:  0.43568840579710144
             precision    recall  f1-score   support

        0.0       0.48      0.43      0.45      1883
        1.0       0.37      0.29      0.33      1751
        2.0       0.44      0.57      0.50      1886

avg / total       0.43      0.44      0.43      5520

[[ 809  457  617]
 [ 473  515  763]
 [ 397  408 1081]]


# with grid search 

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipe_svc = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=1))])

param_range = [ 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    {'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=1)
%time gs = gs.fit(X, Y)

Wall time: 1h 8min 55s


In [27]:
print(gs.best_score_)
print(gs.best_params_)

0.6690944667898685
{'clf__C': 10.0, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}


### gird search with k fold 

In [159]:
model_svc = SVC()

In [160]:
parameters = {
        'C': [ 0.01, 0.1, 1.0, 10.0, 100.0],
        'kernel': ["rbf", "linear"],
        'gamma': [0.01, 0.05, 0.1, 1, 1.5],
        }

clf = GridSearchCV(model_svc, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=3, shuffle=True), 
                   scoring='accuracy',
                   verbose=3, refit=True)


In [ ]:
clf.fit(X_train, y_train)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Acc score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
predictions  = clf.predict(X_test)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_test, predictions, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_test, predictions))

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))